In [56]:
"""



To use this script, make the following preparations:
- Create a working directory
- Inside the working directory, have a single ..._clips.npy file. This file will correspond to which
  character's model the generated data will be used to train.
- Inside the working directory, have a single folder named 'overlay_clips'
- Inside the overlay_clips directory, include one or more ..._clips.npy files of other characters. This will
  be used generate data for the known character's model to differentiate from other characters.
- Inside the working directory, create an empty subdirectory named 'output'
"""

import numpy as np
import librosa
import os
from scipy.io.wavfile import write
import pydub
import random

In [32]:
"""
Specify script parameters
"""
# specify the working directory; note that directory notation must use '/' rather than '\'
wd = 'F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/'



# this will store the known character's clips
known_char_clips = None

# this will store the clips for all other characters
other_char_clips_array = []

# navigate the specified working directory to determine known character and other character clips arrays
wd_contents = os.listdir(wd)
wd_overlays_contents = os.listdir(wd+'overlay_clips')

for item in wd_contents:
    if os.path.isfile(os.path.join(wd, item)):
        print('For known character data: %s\n' % (item))
        
        # load the known character's ...clips.npy file
        with open(wd+item, 'rb') as f:
            known_char_clips = np.load(f)
            
print('For other character data:')
for item in wd_overlays_contents:
    # load the current other character's ...clips.npy file
        with open(wd+'overlay_clips/'+item, 'rb') as f:
            other_char_clips_array.append(np.load(f))
            
        print('\t%s' % (item))
    
            
other_char_clips_array = np.array(other_char_clips_array)

For known character data: id10016_merged_cleaned.wav_clips.npy

For other character data:
	id10130_merged_cleaned.wav_clips.npy
	id10168_merged_cleaned.wav_clips.npy
	id10484_merged_cleaned.wav_clips.npy


In [33]:
print('known_clips.shape:', known_clips.shape)
print('other_char_clips_array.shape:', other_char_clips_array.shape)

known_clips.shape: (1497, 88200)
other_char_clips_array.shape: (3, 1497, 88200)


In [ ]:
# we need to shuffle the indexing order for the known character clips
random.shuffle(array)

In [62]:
to_overlay = [other_char_clips_array[0][0], other_char_clips_array[1][0]]
test = sum(to_overlay)/len(to_overlay)
print(test.shape)

(88200,)


In [63]:
# with open('F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/output/test_overlay.wav', 'wb') as f:
#     np.save(f, test)
loc = 'F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/output/test_overlay.wav'
write(loc, 22050, test)

In [64]:
# # https://stackoverflow.com/questions/42492246/how-to-normalize-the-volume-of-an-audio-file-in-python


<_io.BufferedRandom name='F:/ZaknafeinII_Backup_02-02-22/daea/training_data_generation/id16/output/test_overlay_normalized.wav'>